In [1]:
# Import scrapy and scrapy.crawler 
import scrapy
from scrapy.crawler import CrawlerProcess

import requests
import pandas as pd
import json

import os 
import logging

In [2]:
ville = "Uzes"

# On crée une classe 
class Spider_Booking(scrapy.Spider):

    # On donne un nom a notre spider
    name = "Booking"

    # On lui indique l'URL de la page où commancer le process
    start_urls = ['https://www.booking.com/index.fr.html']

    # On utilise une fonction parse pour lui dire d'aller chercher le formulaire à saisir pour renseigner la ville
    def parse(self, response):

        # On utilise FormRequest pour saisir notre destination (ville) dans le formulaire (id=ss)
        return scrapy.FormRequest.from_response(response, formdata={'ss' : ville},
                                                # Function to be called once logged in
                                                callback=self.scrape_afterSearch)
    def scrape_afterSearch(self, response):
        for hotelName in response.css('.sr_item'):
            yield{
                'cityName' : ville,
                'hotelName' : hotelName.css('.sr-hotel__name::text').get(),
                'URL' : "https://www.booking.com" + hotelName.css('.sr-hotel__title a').attrib["href"],
                'lat&lon_hotel': hotelName.css('.sr_card_address_line a').attrib["data-coords"],
                'score': hotelName.css('.bui-review-score__badge::text').get(),
                'description' : hotelName.css('.hotel_desc::text').get()
            }
            
            
        # Select the NEXT button and store it in next_page
        try:
                next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
                logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback = self.scrape_afterSearch)
            
# Name of the file where the results will be saved
filename = ville + "_hotel_booking.json"

# If file already exists, delete it before crawling
if filename in os.listdir('results/'):
    os.remove('results/' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {'results/' + filename: {"format": "json"},
    }
})

process.crawl(Spider_Booking)
process.start()

2021-08-30 18:58:38 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-08-30 18:58:38 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Linux-5.4.109+-x86_64-with-glibc2.10
2021-08-30 18:58:38 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-08-30 18:58:38 [scrapy.extensions.telnet] INFO: Telnet Password: 7f71b6ada45e26c6
2021-08-30 18:58:38 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-08-30 18:58:38 [scrapy.middleware] INFO: Enab